In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
data_dir = '/home/ubuntu/notebooks/data/expedia'

In [4]:
cols = ['site_name', 'is_mobile', 'is_package', 'channel', 'srch_destination_id', 'user_location_country',
        'hotel_continent']

test = pd.read_csv(os.path.join(data_dir, 'test.csv'), usecols=cols)

cols = ['site_name', 'is_mobile', 'is_package', 'channel', 'srch_destination_id', 'user_location_country',
        'hotel_continent', 'is_booking', 'hotel_cluster']
train = pd.read_csv(os.path.join(data_dir, 'train.csv'), usecols=cols)

# destinations = pd.read_csv(os.path.join(data_dir, 'destinations.csv'))

In [5]:
train = train[train['is_booking']==1].drop(['is_booking'], axis=1)

In [6]:
target = train['hotel_cluster']
train = train.drop(['hotel_cluster'], axis=1)

In [7]:
print test.shape
print train.shape

(2528243, 7)
(3000693, 7)


In [8]:
df = pd.concat([train, test], axis=0)

In [9]:
df.shape

(5528936, 7)

In [10]:
from sklearn.preprocessing import LabelEncoder

cols = ['site_name', 'channel', 'srch_destination_id', 'user_location_country', 'hotel_continent']
cols = ['site_name']
for c in cols:
    df_all_dummy = pd.get_dummies(df[c], prefix=c)
    df = df.drop([c], axis=1)
    df = pd.concat((df, df_all_dummy), axis=1)

In [11]:
c = 'channel'
df_all_dummy = pd.get_dummies(df[c], prefix=c)
df = df.drop([c], axis=1)
df = pd.concat((df, df_all_dummy), axis=1)

In [12]:
from xgboost.sklearn import XGBClassifier

In [13]:
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25, nthread=-1,
                    objective='multi:softprob', subsample=1, colsample_bytree=0.5, seed=0)        

In [14]:
target.shape

(3000693,)

In [15]:
df.values[-3000693:].shape

(3000693, 69)

In [16]:
xgb.fit(df.values[-3000693:], target)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0, learning_rate=0.3, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=25, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [17]:
y_pred = xgb.predict_proba(df.values[:-3000693])

In [28]:
ids = []  
cts = []
id_test = test.index
res = []
for i in range(len(id_test)):
    res.append({'id': i, 'hotel_cluster': ' '.join(str(i) for i in np.argsort(y_pred[i])[::-1][:5])})

In [29]:
res_df = pd.DataFrame(res)

In [30]:
# sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'hotel_cluster'])
res_df.to_csv('simple_xgboost.csv',index=False)

In [19]:
sub.head()

,id,hotel_cluster
0,0,91
1,0,48
2,0,42
3,0,59
4,0,28
